In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

import matplotlib.pyplot as plt

# Generate "Washington Beltway" potential data

In [ ]:
#Generate data according to a "Washington Beltway" potential
#Or sort of like that... just two seperate potential energy minima along the radial distance and uniform in angle
offset = 0.0 #Offset between energy wells in radial distance, in kB*T
mix = 1.0 / (1.0 + np.exp(offset))
print(mix)
r_dist = tfp.distributions.Mixture(tfp.distributions.Categorical(probs=[mix, 1.0-mix]),
                                   [tfp.distributions.Gamma(40.0, rate=40.0),
                                    tfp.distributions.Gamma(160.0, rate=80.0)])
theta_dist = tfp.distributions.Uniform(low=-np.pi, high=np.pi)
wb_dist = tfp.distributions.JointDistributionSequential([r_dist, theta_dist])

In [ ]:
rbins = np.linspace(0.0, 3.0, 100)
plt.hist(r_dist.sample(100000).numpy().flatten(), bins=rbins, histtype='step')
plt.show()

In [ ]:
thetabins = np.linspace(-np.pi, np.pi, 100)
plt.hist(theta_dist.sample(100000).numpy().flatten(), bins=thetabins, histtype='step')
plt.show()

In [ ]:
wb_sample = tf.stack(wb_dist.sample(100000), axis=1).numpy()
plt.hist2d(wb_sample[:, 0], wb_sample[:, 1], bins=[rbins, thetabins])
plt.show()

In [ ]:
x_sample = wb_sample[:, 0]*np.cos(wb_sample[:, 1])
y_sample = wb_sample[:, 0]*np.sin(wb_sample[:, 1])
wb_xy_sample = np.vstack([x_sample, y_sample]).T

xybins = np.linspace(-3.0, 3.0, 100)

plt.hist2d(wb_xy_sample[:, 0], wb_xy_sample[:, 1], bins=[xybins, xybins])
plt.show()

In [ ]:
np.save('wb_rtheta_data', wb_sample)
np.save('wb_xy_data', wb_xy_sample)

In [ ]:
dat_xy = np.load('wb_xy_data.npy')
dat_rtheta = np.load('wb_rtheta_data.npy')

# Create and train VAEs

In [ ]:
import gc
import copy
from libVAE import vae, losses

In [ ]:
#Define VAE parameters
params_vae = {'e_hidden_dim':50,
              'f_hidden_dim':20,
              'd_hidden_dim':50}


In [ ]:
#Define training and plotting routines

def train_vae(model, raw_data,
              num_epochs=100, batch_size=200, anneal_beta_val=None):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, #0.005
                                         beta_1=0.9,
                                         beta_2=0.999,
                                         epsilon=1e-08)
    
    if model.autoregress:
        loss_fn = losses.AutoregressiveLoss(model.decoder,
                                            reduction=tf.keras.losses.Reduction.SUM)
    else:
        loss_fn = losses.ReconLoss(loss_fn=losses.diag_gaussian_loss, activation=None,
                                   reduction=tf.keras.losses.Reduction.SUM)
    
    train_data = tf.data.Dataset.from_tensor_slices(raw_data)
    train_data = train_data.shuffle(buffer_size=3*batch_size).batch(batch_size)
    
    if anneal_beta_val is not None:
        try:
            original_beta = copy.deepcopy(model.beta)
        except AttributeError:
            print("Annealing turned on but model has no beta parameter, turning off.")
            anneal_beta_val = None
    
    for epoch in range(num_epochs):
        
        if anneal_beta_val is not None:
            model.beta = (anneal_beta_val - original_beta)*epoch/(num_epochs - 1)
        
        print("Epoch %i (beta=%f):"%(epoch, model.beta))
        
        for step, batch_train in enumerate(train_data):
                        
            for ametric in model.metrics:
                ametric.reset_states()
            
            with tf.GradientTape() as tape:
                reconstructed = model(batch_train, training=True)
                loss = loss_fn(batch_train, reconstructed) / batch_train.shape[0]
                loss += sum(model.losses)
                
            grads = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
            if step%100 == 0:
                print('\tStep %i: loss=%f, model_loss=%f, kl_div=%f, reg_loss=%f'
                      %(step, loss, sum(model.losses), model.metrics[0].result(), model.metrics[1].result()))
            
            if tf.math.is_nan(loss):
                raise ValueError('Loss is NaN.')
                
            gc.collect()


# Train in 1D
Train with x and y coordinates, then r and theta, with prior flow VAE, prior flow with regular autoregression, then full flow VAE.

In [ ]:
#First with x and y data for all models
vae_xy_1D = vae.PriorFlowVAE((2,), 1, include_vars=True, autoregress=False, **params_vae)
vae_xy_1D_auto = vae.PriorFlowVAE((2,), 1, include_vars=True, autoregress=True, **params_vae)
vae_xy_1D_full = vae.FullFlowVAE((2,), 1, **params_vae)

for model, fname in [[vae_xy_1D, 'vae_xy_1D'],
                     [vae_xy_1D_auto, 'vae_xy_1D_auto'],
                     [vae_xy_1D_full, 'vae_xy_1D_full']]:
    model.beta = 0.0
    train_vae(model, dat_xy, num_epochs=20, anneal_beta_val=1.0)
    train_vae(model, dat_xy, num_epochs=80)
    model.save_weights(fname+'.h5')


In [ ]:
#Next with r and theta data for all models
#Make sure to specify that theta is periodic (will use -pi to pi)
#Unfortunately, no way to set bounds for r from here...
#But default of -10 to 10 should be alright because contains range for r
vae_rtheta_1D = vae.PriorFlowVAE((2,), 1, include_vars=True, autoregress=False, periodic_dof_inds=[1], **params_vae)
vae_rtheta_1D_auto = vae.PriorFlowVAE((2,), 1, include_vars=True, autoregress=True, periodic_dof_inds=[1], **params_vae)
vae_rtheta_1D_full = vae.FullFlowVAE((2,), 1, periodic_dof_inds=[1], **params_vae)

for model, fname in [[vae_rtheta_1D, 'vae_rtheta_1D'],
                     [vae_rtheta_1D_auto, 'vae_rtheta_1D_auto'],
                     [vae_rtheta_1D_full, 'vae_rtheta_1D_full']]:
    model.beta = 0.0
    train_vae(model, dat_rtheta, num_epochs=20, anneal_beta_val=1.0)
    train_vae(model, dat_rtheta, num_epochs=80)
    model.save_weights(fname+'.h5')


# Train in 2D
Train with x and y coordinates, then r and theta, with prior flow VAE, prior flow with regular autoregression, then full flow VAE.

In [ ]:
#First with x and y data for all models
vae_xy_2D = vae.PriorFlowVAE((2,), 2, include_vars=True, autoregress=False, **params_vae)
vae_xy_2D_auto = vae.PriorFlowVAE((2,), 2, include_vars=True, autoregress=True, **params_vae)
vae_xy_2D_full = vae.FullFlowVAE((2,), 2, **params_vae)

for model, fname in [[vae_xy_2D, 'vae_xy_2D'],
                     [vae_xy_2D_auto, 'vae_xy_2D_auto'],
                     [vae_xy_2D_full, 'vae_xy_2D_full']]:
    model.beta = 0.0
    train_vae(model, dat_xy, num_epochs=20, anneal_beta_val=1.0)
    train_vae(model, dat_xy, num_epochs=80)
    model.save_weights(fname+'.h5')


In [ ]:
#Next with r and theta data for all models
vae_rtheta_2D = vae.PriorFlowVAE((2,), 2, include_vars=True, autoregress=False, periodic_dof_inds=[1], **params_vae)
vae_rtheta_2D_auto = vae.PriorFlowVAE((2,), 2, include_vars=True, autoregress=True, periodic_dof_inds=[1], **params_vae)
vae_rtheta_2D_full = vae.FullFlowVAE((2,), 2, periodic_dof_inds=[1], **params_vae)

for model, fname in [[vae_rtheta_2D, 'vae_rtheta_2D'],
                     [vae_rtheta_2D_auto, 'vae_rtheta_2D_auto'],
                     [vae_rtheta_2D_full, 'vae_rtheta_2D_full']]:
    model.beta = 0.0
    train_vae(model, dat_rtheta, num_epochs=20, anneal_beta_val=1.0)
    train_vae(model, dat_rtheta, num_epochs=80)
    model.save_weights(fname+'.h5')
